DB name: pyspark-udemy-course-db
DB password: abcde12345

database-2.cgewr2fnx8mr.eu-west-2.rds.amazonaws.com

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType

spark = (
    SparkSession.builder
        .appName("ETL")
        # .config("spark.jars", "/postgresql-42.7.0.jar") 
        .config('spark.driver.extraClassPath', "/postgresql-42.7.0.jar")
        .getOrCreate()
)

/usr/local/lib/python3.8/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/09 15:37:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Read and transform

In [2]:
df = spark.read.text("../data/WordData_etl.txt")

In [3]:
def split(x: str):
    return x.split(" ")

split_udf = F.udf(lambda x: split(x), ArrayType(StringType()))

df2 = (
    df
        .withColumn("splitted", F.split("value", " "))
        .select("splitted", F.explode(F.col("splitted")))
        .groupBy('col')
        .count()
)

df2.show()

+-----------+-----+
|        col|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows



Load to DB

In [5]:
driver = "org.postgres.Driver"
url = "database-2.cgewr2fnx8mr.eu-west-2.rds.amazonaws.com/"
table = "felix_pyspark.WordCount"
user = "postgres"
pw = "abcde12345"

(
    df2.write.format("jdbc")
        .option("driver", driver)
        .option("url", url)
        .option("dbtable", table)
        .option("mode", "append")
        .option("user", user)
        .option("password", pw)
).save()

Py4JJavaError: An error occurred while calling o62.save.
: java.sql.SQLException: No suitable driver
	at java.sql.DriverManager.getDriver(DriverManager.java:315)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:109)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:109)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:47)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.Ex

23/12/09 16:09:39 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1770197 ms exceeds timeout 120000 ms
23/12/09 16:09:39 WARN SparkContext: Killing executors is not supported by current scheduler.
23/12/09 16:09:39 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$